In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter("ignore")

In [2]:
df = pd.read_csv("Churn_analyzed.csv")
df.head()

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,619,France,0,6.480741,2,0.00,1,1,1,101348.88,1
1,1,608,Spain,0,6.403124,1,83807.86,1,0,1,112542.58,0
2,2,502,France,0,6.480741,8,159660.80,3,1,0,113931.57,1
3,3,699,France,0,6.244998,1,0.00,2,0,0,93826.63,0
4,4,850,Spain,0,6.557439,2,125510.82,1,1,1,79084.10,0


In [19]:
X = df.drop(columns = ["Exited", "Unnamed: 0"])
X = pd.get_dummies(P, drop_first = True, dtype = int)
y = df["Exited"]

# Train Test Split

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [15]:
sc = StandardScaler()

***Finding Best Random State***

In [20]:
train = []
test = []
train_acc = []
test_acc = []
cv = []
mae = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(P, y, test_size = 0.2, random_state = i)
    X_train.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]] = sc.fit_transform(X_train.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]])
    X_test.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]] = sc.transform(X_test.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]])
    model = LogisticRegression().fit(X_train, y_train)
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)

    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)

    train_score = accuracy_score(y_train, pred_train)
    test_score = accuracy_score(y_test, pred_test)

    mae_score = mean_absolute_error(y_test, pred_test)
    cv_score = cross_val_score(model, X_train, y_train, cv = 5).mean()

    train.append(train_accuracy)
    test.append(test_accuracy)
    train_acc.append(train_score)
    test_acc.append(test_score)
    cv.append(cv_score)
    mae.append(mae_score)

ff = pd.DataFrame({
    "Train Model": train,
    "Test Model": test,
    "Train acc": train_acc,
    "Test acc": test_acc,
    "CV": cv,
    "MAE": mae
})

In [21]:
ff.sort_values(by = "MAE", ascending = True)

,Train Model,Test Model,Train acc,Test acc,CV,MAE
39,0.808875,0.8330,0.808875,0.8330,0.806875,0.1670
54,0.809500,0.8270,0.809500,0.8270,0.809000,0.1730
40,0.809875,0.8250,0.809875,0.8250,0.807625,0.1750
93,0.810250,0.8240,0.810250,0.8240,0.809875,0.1760
4,0.810500,0.8230,0.810500,0.8230,0.810375,0.1770
...,...,...,...,...,...,...
90,0.817125,0.8005,0.817125,0.8005,0.816875,0.1995
49,0.817250,0.8000,0.817250,0.8000,0.815500,0.2000
78,0.816000,0.7995,0.816000,0.7995,0.814125,0.2005
84,0.816875,0.7945,0.816875,0.7945,0.816625,0.2055


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 39)
scaler = StandardScaler()
X_train.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]] = scaler.fit_transform(X_train.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]])
X_test.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]]= scaler.transform(X_test.loc[:, ["CreditScore", "Age", "Balance", "EstimatedSalary"]])

# ***GridSearchCV - SVC***

In [ ]:
svc_estimator = SVC()

svc_param_grid = {"C": [0.001, 0.1, 0, 10, 100], "kernel": ["linear", "poly", "sigmoid", "rbf"]}

svc_grid = GridSearchCV(svc_estimator, svc_param_grid, cv = 5, scoring = "accuracy")

svc_grid.fit(X_train, y_train)

svc_grid.best_params_

# ***KNeighborsClassifier***

In [25]:
knn_estimator = KNeighborsClassifier()

knn_param_grid = {"n_neighbors": list(range(1, 20))}

knn_grid = GridSearchCV(knn_estimator, knn_param_grid, cv = 5, scoring = "accuracy")

knn_grid.fit(X_train, y_train)

knn_grid.best_params_

{'n_neighbors': 11}

# ***DecisionTreeClassifier***

In [26]:
decision_estimator = DecisionTreeClassifier()

decision_param_grid = {"max_depth": list(range(1, 20)), "criterion": ["gini", "entropy"]}

decision_grid = GridSearchCV(decision_estimator, decision_param_grid, cv = 5, scoring = "accuracy")

decision_grid.fit(X_train, y_train)

decision_grid.best_params_

{'criterion': 'entropy', 'max_depth': 6}

# ***RandomForestClassifier***

In [29]:
random_estimator = RandomForestClassifier()

random_param_grid = {"n_estimators": list(range(1, 20))}

random_grid = GridSearchCV(random_estimator, random_param_grid, cv = 5, scoring = "accuracy")

random_grid.fit(X_train, y_train)

random_grid.best_params_

{'n_estimators': 17}

# ***AdaBoostClassifier***

In [30]:
ada_estimator = AdaBoostClassifier()

ada_param_grid = {"n_estimators": list(range(1, 20))}

ada_grid = GridSearchCV(ada_estimator, ada_param_grid, cv = 5, scoring = "accuracy")

ada_grid.fit(X_train, y_train)

ada_grid.best_params_

{'n_estimators': 13}

# ***GradientBoostingClassifier***

In [31]:
gradient_estimator = GradientBoostingClassifier()

gradient_param_grid = {"n_estimators": list(range(1, 20)), "learning_rate" : [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

gradient_grid = GridSearchCV(gradient_estimator, gradient_param_grid, cv = 5, scoring = "accuracy")

gradient_grid.fit(X_train, y_train)

gradient_grid.best_params_

{'learning_rate': 0.4, 'n_estimators': 15}

# ***XGBClassifier***

In [32]:
xgb_estimator = XGBClassifier()

xgb_param_grid = {"n_estimators": list(range(5, 20)), "learning_rate" : [0, 0.1, 0.2, 0.3, 0.4, 0.5],
                 "gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5]}

xgb_grid = GridSearchCV(xgb_estimator, xgb_param_grid, cv = 5, scoring = "accuracy")

xgb_grid.fit(X_train, y_train)

xgb_grid.best_params_

{'gamma': 0.4, 'learning_rate': 0.3, 'n_estimators': 15}